In [ ]:
import os
import open3d as o3d
import numpy as np
from sklearn.linear_model import RANSACRegressor

mini_dataset_dir = './pointclouds_mini'
output_dir = './output_pointclouds'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over all .pcd files in the directory
for pcd_file in os.listdir(mini_dataset_dir):
    if pcd_file.endswith('.pcd'):
        pcd_file_path = os.path.join(mini_dataset_dir, pcd_file)
        point_cloud = o3d.io.read_point_cloud(pcd_file_path)
        points = np.asarray(point_cloud.points)
        
        if points.shape[0] == 0:
            print(f"Skipping empty point cloud: {pcd_file}")
            continue
        
        X = points[:, :2]  # x and y coordinates
        y = points[:, 2]   # z coordinate (height)

        # Define the RANSAC regressor
        ransac = RANSACRegressor()

        # Fit the model
        ransac.fit(X, y)

        # Predict inliers and outliers
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)

        # Separate ground points (inliers) and non-ground points (outliers)
        ground_points = points[inlier_mask]
        non_ground_points = points[outlier_mask]

        # Create point cloud objects for ground and non-ground points
        ground_pcd = o3d.geometry.PointCloud()
        ground_pcd.points = o3d.utility.Vector3dVector(ground_points)

        non_ground_pcd = o3d.geometry.PointCloud()
        non_ground_pcd.points = o3d.utility.Vector3dVector(non_ground_points)

        # Save the ground and non-ground points to new .pcd files
        ground_output_path = os.path.join(output_dir, f"ground_{pcd_file}")
        non_ground_output_path = os.path.join(output_dir, f"non_ground_{pcd_file}")

        o3d.io.write_point_cloud(ground_output_path, ground_pcd)
        o3d.io.write_point_cloud(non_ground_output_path, non_ground_pcd)

        print(f"Processed {pcd_file}: saved ground points to {ground_output_path} and non-ground points to {non_ground_output_path}")

In [ ]:
import pcl
import numpy as np
import matplotlib.pyplot as plt

def point_cloud_to_image(pcd):
    # Convert point cloud to numpy array
    points = np.asarray(pcd)
    
    # For simplicity, project the points onto the XY plane (ignoring Z for a 2D view)
    x = points[:, 0]
    y = points[:, 1]
    
    # Create a scatter plot for visualization
    plt.figure(figsize=(10, 10))
    plt.scatter(x, y, s=1, color="blue")  # Plot points
    plt.xlim([-50, 50])  # Adjust according to your data
    plt.ylim([-50, 50])
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis('off')
    return plt

def save_frame_as_image(frame, filename):
    plt = point_cloud_to_image(frame)
    plt.savefig(filename, dpi=300)
    plt.close()


In [ ]:
import os
import pcl

output_folder = './ground_frames'
os.makedirs(output_folder, exist_ok=True)
pcd_directory = "./output_pointclouds"

for i in range(3605, 3732):  # 100 frames, 10 seconds at 10Hz
    pcd_file = f'ground_{i:06d}.pcd'
    file_path = os.path.join(pcd_directory, pcd_file)

    pcd = pcl.load(file_path)  # Modify based on your file structure
    filename = os.path.join(output_folder, f"frame_{i:06d}.png")
    save_frame_as_image(pcd, filename)


In [ ]:
import cv2
import os

# Set the path to the folder containing the PNG files
image_folder = './ground_frames'
output_video = 'ground_minicloud.avi'

# Get the list of PNG files and sort them
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort()  # Ensure images are sorted in order

# Read the first image to get the dimensions for the video
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Create a video writer object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change this codec
video = cv2.VideoWriter(output_video, fourcc, 10, (width, height))  # 30 FPS

# Loop through all the images and add them to the video
for image in images:
    img_path = os.path.join(image_folder, image)
    img = cv2.imread(img_path)
    video.write(img)

# Release the video writer object
video.release()

print(f"Video saved as {output_video}")
